<a href="https://colab.research.google.com/github/veekaybee/viberary/blob/bert_notebook/08_generate_embeddings_bert_sentence_transformer_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

!{sys.executable} -W ignore:DEPRECATION -m pip install --quiet duckdb==0.7.1 \
duckdb-engine \
watermark \
jupysql \
sqlalchemy \
python-snappy \
pyarrow \
memray \
pandas  \
ipywidgets  \
matplotlib \
gensim \
nltk \
plotly \
redis==4.5.3 \
jupyter-black \
sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.6/238.6 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.0/269.0 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.9 MB/s eta 0:00:00

In [2]:
# Autoformat cells on run
import jupyter_black
import pandas as pd

jupyter_black.load()

In [3]:
import duckdb
import re
import pandas as pd
import string
from time import time

In [4]:
# set log level for model training
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(
    format="%(levelname)s - %(asctime)s: %(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO,
)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
embeddings = pd.read_csv("/content/drive/My Drive/Colab Notebooks/embedding_data.csv")

In [8]:
embeddings

,Unnamed: 0,title,sentence
0,0,W.C. Fields: A Life on Film,w c fields a life on film
1,1,Good Harbor,good harbor anita diamant s international best...
2,2,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",the unschooled wizard sun wolf and starhawk ...
3,3,Best Friends Forever,best friends forever addie downs and valerie a...
4,4,Runic Astrology: Starcraft and Timekeeping in ...,runic astrology starcraft and timekeeping in ...
...,...,...,...
2360650,2360650,"This Sceptred Isle, Vol. 10: The Age of Victor...",this sceptred isle vol the age of victor...
2360651,2360651,Sherlock Holmes and the July Crisis,sherlock holmes and the july crisis sir arthur...
2360652,2360652,The Children's Classic Poetry Collection,the children s classic poetry collection gathe...
2360653,2360653,"101 Nights: Volume One (101 Nights, #1-3)",nights volume one nights volu...


In [13]:
corpus = embeddings["sentence"].tolist()

In [14]:
from sentence_transformers import SentenceTransformer, util

#  A common value for BERT & Co. are 512 word pieces, which corresponde to about 300-400 words (for English).
# Longer texts than this are truncated to the first x word pieces.
# By default, the provided methods use a limit fo 128 word pieces, longer inputs will be truncated
# the runtime and the memory requirement grows quadratic with the input length - we'll have to play around with this

# Change the length to 200
model = SentenceTransformer("all-MiniLM-L6-v2")
model.max_seq_length = 200


corpus_embeddings = model.encode(
    corpus, show_progress_bar=True,device='cuda', convert_to_numpy=False
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


Batches:   0%|          | 0/73771 [00:00<?, ?it/s]

In [15]:
embedding_list = [element.tolist() for element in corpus_embeddings]

In [16]:
embedding_list[1]

[-0.009158187545835972,
 -0.06342962384223938,
 0.04988787695765495,
 0.0510813444852829,
 -0.03709794580936432,
 0.039988309144973755,
 0.06472879648208618,
 -0.06210295483469963,
 -0.06761530786752701,
 -0.015654467046260834,
 -0.008045890368521214,
 0.006510134320706129,
 -0.06701060384511948,
 0.032156918197870255,
 -0.022840328514575958,
 0.09047019481658936,
 -0.001248328946530819,
 -0.009074331261217594,
 -0.013187597505748272,
 0.0387604795396328,
 -0.06587356328964233,
 -0.006924374494701624,
 0.04098755493760109,
 -0.01678481511771679,
 -0.0388915091753006,
 -0.023469120264053345,
 0.03800095245242119,
 0.033769458532333374,
 -0.0671144425868988,
 0.01039760559797287,
 -0.13183170557022095,
 0.026348350569605827,
 -0.0001357018918497488,
 0.0028514135628938675,
 0.06357114017009735,
 0.060107361525297165,
 0.021407637745141983,
 0.014468754641711712,
 0.023105142638087273,
 0.008717063814401627,
 -0.03225822001695633,
 -0.04530007019639015,
 -0.005124491639435291,
 0.03960687

In [18]:
embeddings["embeddings"] = embedding_list

In [19]:
sentence_embeddings = embeddings[["title", "embeddings"]]

In [21]:
sentence_embeddings

# sentence_embeddings.loc[sentence_embeddings["embeddings"]] = sentence_embeddings[
#     ["embeddings"]
# ].to_numpy()

,title,embeddings
0,W.C. Fields: A Life on Film,"[-0.09234853088855743, -0.006951830815523863, ..."
1,Good Harbor,"[-0.009158187545835972, -0.06342962384223938, ..."
2,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","[-0.019842753186821938, -0.03624306991696358, ..."
3,Best Friends Forever,"[-0.06455371528863907, -0.021649178117513657, ..."
4,Runic Astrology: Starcraft and Timekeeping in ...,"[-0.0018127849325537682, 0.07648871093988419, ..."
...,...,...
2360650,"This Sceptred Isle, Vol. 10: The Age of Victor...","[-0.010960440151393414, -0.023233134299516678,..."
2360651,Sherlock Holmes and the July Crisis,"[-0.03523332625627518, -0.0030151745304465294,..."
2360652,The Children's Classic Poetry Collection,"[0.007812583819031715, -0.05686502531170845, 5..."
2360653,"101 Nights: Volume One (101 Nights, #1-3)","[0.01172102615237236, -0.11345934122800827, 0...."


In [33]:
sentence_embeddings.to_csv("sentence_embeddings.csv")

KeyboardInterrupt: ignored

In [37]:
sentence_embeddings['idx'] = sentence_embeddings.index

<ipython-input-37-bb8e87a91e86>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentence_embeddings['idx'] = sentence_embeddings.index


In [41]:
sentence_embeddings

,title,embeddings,idx
0,W.C. Fields: A Life on Film,"[-0.09234853088855743, -0.006951830815523863, ...",0
1,Good Harbor,"[-0.009158187545835972, -0.06342962384223938, ...",1
2,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","[-0.019842753186821938, -0.03624306991696358, ...",2
3,Best Friends Forever,"[-0.06455371528863907, -0.021649178117513657, ...",3
4,Runic Astrology: Starcraft and Timekeeping in ...,"[-0.0018127849325537682, 0.07648871093988419, ...",4
...,...,...,...
2360650,"This Sceptred Isle, Vol. 10: The Age of Victor...","[-0.010960440151393414, -0.023233134299516678,...",2360650
2360651,Sherlock Holmes and the July Crisis,"[-0.03523332625627518, -0.0030151745304465294,...",2360651
2360652,The Children's Classic Poetry Collection,"[0.007812583819031715, -0.05686502531170845, 5...",2360652
2360653,"101 Nights: Volume One (101 Nights, #1-3)","[0.01172102615237236, -0.11345934122800827, 0....",2360653


In [42]:
import numpy as np

embedding_dict = dict(
    zip(
        sentence_embeddings["idx"],
        sentence_embeddings["embeddings"]
    )
)

In [ ]:
import json


with open('embedding_dict.json', 'w') as fp:
    json.dump(embedding_dict, fp)

#data = json.load(open(filename))

In [43]:
first_value = next(iter(embedding_dict.items()))
first_value

(0,
 [-0.09234853088855743,
  -0.006951830815523863,
  -0.05996650829911232,
  -0.009789670817553997,
  0.032347824424505234,
  0.06075367331504822,
  0.08125866204500198,
  0.043228231370449066,
  0.05158374086022377,
  -0.014283852651715279,
  0.025735978037118912,
  -0.03713679686188698,
  -0.03816591575741768,
  0.07240444421768188,
  -0.08822837471961975,
  0.05091795697808266,
  -0.03594037517905235,
  -0.008459102362394333,
  -0.03531213849782944,
  0.02569682151079178,
  -0.05043340474367142,
  -0.02963639423251152,
  0.008075146935880184,
  0.010749317705631256,
  -0.0898258164525032,
  -0.0012774348724633455,
  -0.03546484187245369,
  0.0639660507440567,
  -0.07411997765302658,
  -0.058101944625377655,
  -0.040556613355875015,
  0.14181597530841827,
  0.012417006306350231,
  0.028205035254359245,
  0.0494011826813221,
  0.03820422664284706,
  -0.0006648338166996837,
  0.004045492969453335,
  -0.032919589430093765,
  0.018863964825868607,
  -0.0951518714427948,
  -0.0835943669